# 1. Tratamento dos dados e criacao da coluna target

## Importar bibliotecas

In [2]:
# ! pip install xlrd
# ! pip install openpyxl
# ! pip install pydantic
# ! pip install joblib

In [3]:
import numpy as np
import pandas as pd

import joblib

In [4]:
def carregar_dados_aquisicao():
    df = joblib.load("../out/result_aquisicao.joblib")
    print("arquivo joblib carregado com sucesso")

    return df


In [5]:
def rename_columns(df):
    df = df.copy()
    columns_rename={
        "Ativo Total": "ativo_total", 
        "Ativo Circulante": "ativo_circ",
        "Ativo Realizável a Longo Prazo": "ativo_permanente",
        "Passivo Circulante": "passivo_circ", 
        "Passivo Não Circulante": "passivo_nao_circ",
        "Patrimônio Líquido": "patrimonio_liq", 
        "Receita Líquida de Vendas e/ou Serviços": "receita_liq", 
        "Custo de Bens e/ou Serviços Vendidos": "custo_bens_servicos", 
        "Lucro/Prejuízo do Período": "lucro_op", 
        "Outros Ativos Circulantes": "outros_ativos_circ", 
        "Caixa e Equivalentes de Caixa": "caixa_equiv_caixa",
        "Outros Ativos Não Circulantes": "outros_ativos_nao_circ", 
        "Imobilizado": "imobilizado", 
        "Intangível": "intangivel",
        "Passivo Total": "passivo_total", 
        "Capital Social Realizado": "capital_social",
        "Resultado Bruto": "resultado_bruto",
        "Despesas Gerais e Administrativas": "despesas_gerais", 
        "Financeiras": "financeiras",
        "Resultado Antes Tributação/Participações": "resultado_antes_trib"}

    df.rename(columns=columns_rename, inplace=True)
    df.iloc[:,:-1] = df.iloc[:,:-1].astype(float)
    return df

In [6]:
def select_columns_for_calc(dfs):
    
    colunas_para_calculos = [
        "Ativo Total", 
        "Ativo Realizável a Longo Prazo",
        "Ativo Circulante",
        "Passivo Circulante", 
        "Passivo Não Circulante", 
        "Patrimônio Líquido", 
        "Receita Líquida de Vendas e/ou Serviços", 
        "Lucro/Prejuízo do Período", 
        "Adj Close",
        "IBOV"]

    columns_replace={
        "Ativo Total": "ativo_total", 
        "Ativo Circulante": "ativo_circ",
        "Ativo Realizável a Longo Prazo": "ativo_permanente",
        "Passivo Circulante": "passivo_circ", 
        "Passivo Não Circulante": "passivo_nao_circ", 
        "Patrimônio Líquido": "patrimonio_liq", 
        "Receita Líquida de Vendas e/ou Serviços": "receita_liq", 
        "Lucro/Prejuízo do Período": "lucro_op", 
        "Adj Close": "cotacao"}

    for code in dfs.keys():
        df_c = dfs[code]
        df_c = df_c[colunas_para_calculos]
        df_c = df_c.rename(columns=columns_replace)
        df_c.iloc[:,:-1] = df_c.iloc[:,:-1].astype(float)

        dfs[code] = df_c

    print("colunas para calculo selecionadas com sucesso")

    return dfs


In [7]:
def calcular_dupont(dfs_in):
    dfs = dfs_in.copy()
    for code in dfs.keys():
        df = dfs[code]

        af = df["ativo_total"] / df["patrimonio_liq"]
        ra = df["lucro_op"] / df["ativo_total"]
        # ga = df["receita_liq"] / df["ativo_total"]
        # ml = df["lucro_op"] / df["receita_liq"]
        rpl = af * ra
        roe = df["lucro_op"] / df["patrimonio_liq"]
        # roa = ga * ml

        df["RPL"] = rpl
        df["ROE"] = roe
        # df["ROA"] = roa
        # df["GA"] = ga
        # df["ML"] = ml
        df["AF"] = af
        df["RA"] = ra

        dfs[code] = df

    print("indicadores dupont criados com sucesso")

    return dfs


In [8]:
def calcular_indicadores_financeiros(dfs_in):

    dfs = dfs_in.copy()

    for code in dfs.keys():

        df = dfs[code]

        # Indicadores de endividamento
        part_cap_terceiros = (df["passivo_circ"] + df["passivo_nao_circ"]) / df["patrimonio_liq"]
        comp_endividamento = df["passivo_circ"] / (df["passivo_circ"] + df["passivo_nao_circ"])
        imob_patrimonio_liq = df["ativo_permanente"] / df["patrimonio_liq"]
        imob_recursos_n_corr = df["ativo_permanente"] / (df["patrimonio_liq"] + df["passivo_nao_circ"])

        df["part_cap_terceiros"] = part_cap_terceiros
        df["comp_endividamento"] = comp_endividamento
        df["imob_patrimonio_liq"] = imob_patrimonio_liq
        df["imob_recursos_n_corr"] = imob_recursos_n_corr

        # Indicadores de liquidez

        liq_corrente = df["ativo_circ"] / df["passivo_circ"]

        df["liq_corrente"] = liq_corrente

        dfs[code] = df

    print("indicadores financeiros criados com sucesso")

    return dfs


## Criacao de rotulos - coluna target

## Criar coluna resultado baseado em cotacao e IBOV

- cotacao e Ibov, considerar valores do proximo trimestre 
- Considerar comprar (1): variacao da cotacao acima da variacao do IBOV
- Considerar Vender (0): variacao da cotacao de 2% abaixo da variacao do IBOV

In [25]:
def criar_coluna_decisao(dfs_in): 
    
    dfs = dfs_in.copy()

    for code in dfs.keys():
        df = dfs[code]
        
        df = df.sort_index()

        df["cotacao_var"] = df["Adj Close"].shift(-1) / df["Adj Close"] - 1
        df["IBOV_var"] = df["IBOV"].shift(-1) / df["IBOV"] - 1
        df["resultado"] = df["cotacao_var"] - df["IBOV_var"]

        condicoes = [
            (df["resultado"] > 0 ), 
            (df["resultado"] < -0.02)
        ]
        valores = [1, 0]

        df["decisao"] = np.select(condicoes, valores)

        df = df.drop(["Adj Close", "cotacao_var", "IBOV", "IBOV_var", "resultado"], axis=1)

        dfs[code] = df

    print("coluna decisao/target criada com sucesso")

    return dfs


In [10]:
def remover_colunas_de_calculo(dfs_in):

   colunas_remover = ['ativo_total', 'ativo_permanente', 'ativo_circ', 'passivo_circ',
       'passivo_nao_circ', 'patrimonio_liq', 'receita_liq', 'lucro_op']

   dfs = dfs_in.copy()

   for code in dfs.keys():
      df = dfs[code]
      df = df.drop(colunas_remover, axis=1)
      dfs[code] = df
    
   print("Colunas para calculo removidas com sucesso")

   return dfs

In [11]:
def transformar_indic_em_porcentagem_delta(dfs_in):
    dfs = dfs_in.copy()

    for code in dfs.keys():
        fundamento = dfs[code]

        fundamento.sort_index()


        for coluna in fundamento.columns:
            if coluna != "decisao":
                condicoes = [
                    (fundamento[coluna].shift(1) > 0) & (fundamento[coluna] < 0),
                    (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] > 0),
                    (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] < 0),
                    (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] > 0),
                    (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] < 0),
                    (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] == 0),
                ]
                valores = [
                    -1,
                    1,
                    (abs(fundamento[coluna].shift(1)) - abs(fundamento[coluna])) / abs(fundamento[coluna].shift(1)),
                    1,
                    -1,
                    1,
                ]
                fundamento[coluna] = np.select(condicoes, valores, default=fundamento[coluna] / fundamento[coluna].shift(1) - 1)
    
        # fundamento["codigo"] = code
        
    dfs[code] = fundamento


    return dfs

In [12]:
def juntar_dataframes(dfs_in):
    copia_fund = dfs_in.copy()
    bd = pd.DataFrame()
    for code in copia_fund:
        copia_fund[code] = copia_fund[code][1:-1]
        copia_fund[code] = copia_fund[code].reset_index(drop=True)
        bd = bd.append(copia_fund[code])
    bd.reset_index(drop=True, inplace=True)
    print(f"Dataframe unico criado com sucesso. Registros: {len(bd)}")
    return bd


## Salvar dataframe em joblib

In [13]:
def salvar_df_in_joblib(df, file_name: str = "../out/tratamento_target.joblib"):

    joblib.dump(df, file_name)

    print("arquivo joblib de df decisao criado com sucesso")


# Execucao

In [26]:
def main():

    dfs = carregar_dados_aquisicao()

    # dfs = select_columns_for_calc(dfs)

    # dfs = calcular_dupont(dfs)

    # dfs = calcular_indicadores_financeiros(dfs)

    dfs = criar_coluna_decisao(dfs)


    # dfs = remover_colunas_de_calculo(dfs)

    dfs = transformar_indic_em_porcentagem_delta(dfs)

    # juntar os dataframes das empresas em um
    bd = juntar_dataframes(dfs)

    bd = rename_columns(bd)

    # salvar em arquivo joblib
    salvar_df_in_joblib(bd, file_name="../out/trat_target_col_ibov.joblib")
    
    return bd
    
bd = main()


arquivo joblib carregado com sucesso
coluna decisao/target criada com sucesso
Dataframe unico criado com sucesso. Registros: 1852
arquivo joblib de df decisao criado com sucesso


In [ ]:
bd.columns


Index(['ativo_total', 'ativo_circ', 'caixa_equiv_caixa', 'outros_ativos_circ',
       'ativo_permanente', 'outros_ativos_nao_circ', 'imobilizado',
       'intangivel', 'passivo_total', 'passivo_circ', 'passivo_nao_circ',
       'patrimonio_liq', 'capital_social', 'receita_liq',
       'custo_bens_servicos', 'resultado_bruto', 'despesas_gerais',
       'financeiras', 'resultado_antes_trib', 'lucro_op', 'decisao'],
      dtype='object')

In [ ]:
# for k in bd.keys():
#     print(f"{k} {bd[k].shape}")

bd["ITSA4"].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 33 entries, 2020-12-31 to 2012-12-31
Freq: -1Q-DEC
Data columns (total 24 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Ativo Total                               33 non-null     float64
 1   Ativo Circulante                          33 non-null     float64
 2   Caixa e Equivalentes de Caixa             33 non-null     float64
 3   Outros Ativos Circulantes                 33 non-null     float64
 4   Ativo Realizável a Longo Prazo            33 non-null     float64
 5   Outros Ativos Não Circulantes             15 non-null     float64
 6   Imobilizado                               33 non-null     float64
 7   Intangível                                33 non-null     float64
 8   Passivo Total                             33 non-null     float64
 9   Passivo Circulante                        33 non-null     float64
 10  Emprés